## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-18-51-51 +0000,nypost,Eric Adams’ wildest moments as NYC’s mayor: ‘H...,https://nypost.com/2025/09/28/us-news/eric-ada...
1,2025-09-28-18-51-11 +0000,nyt,"Shooting at Mormon Church in Grand Blanc, Mich...",https://www.nytimes.com/live/2025/09/28/us/gra...
2,2025-09-28-18-50-23 +0000,nyt,Trump Again Focuses on Portland as an Avatar o...,https://www.nytimes.com/2025/09/28/us/portland...
3,2025-09-28-18-46-27 +0000,nyt,"Bar Shooting in Southport, N.C., Was ‘Targeted...",https://www.nytimes.com/2025/09/28/us/north-ca...
4,2025-09-28-18-44-00 +0000,wsj,Trump to attend Pentagon meeting with top comm...,https://www.wsj.com/politics/national-security...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2372/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,30
10,shooting,11
84,new,9
45,dead,9
12,church,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,77
23,2025-09-28-17-42-29 +0000,wapo,"At least one dead, nine injured in Michigan ch...",https://www.washingtonpost.com/nation/2025/09/...,61
8,2025-09-28-18-28-32 +0000,bbc,At least one dead and nine injured in shooting...,https://www.bbc.com/news/articles/ceq2vd15glwo...,61
164,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,61
42,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...,60


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,77,2025-09-28-15-55-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
8,61,2025-09-28-18-28-32 +0000,bbc,At least one dead and nine injured in shooting...,https://www.bbc.com/news/articles/ceq2vd15glwo...
75,57,2025-09-28-13-48-38 +0000,nypost,Eric Adams leaves door open to dropping out of...,https://nypost.com/2025/09/28/us-news/eric-ada...
42,30,2025-09-28-16-05-59 +0000,nypost,Protesters clash with ICE agents as Trump depl...,https://nypost.com/2025/09/28/us-news/proteste...
83,30,2025-09-28-13-13-51 +0000,bbc,Building of three new towns will start before ...,https://www.bbc.com/news/articles/cly1geen679o...
91,29,2025-09-28-12-12-08 +0000,cbc,"Russia pounds Kyiv, other regions in mass dron...",https://www.cbc.ca/news/world/russia-mass-dron...
181,28,2025-09-27-19-49-11 +0000,nypost,"More drones over Denmark, Lithuania stoke fear...",https://nypost.com/2025/09/27/world-news/more-...
144,27,2025-09-28-01-46-27 +0000,nypost,"Stampede at India political rally kills 36, in...",https://nypost.com/2025/09/27/world-news/stamp...
139,27,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
50,25,2025-09-28-15-44-01 +0000,nypost,Accused Southport shooter Nigel Edge once esco...,https://nypost.com/2025/09/28/us-news/accused-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
